# Improving Classification with LLMs using Prompt Learning

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/phx.jpeg" width="800">

In this notebook we will leverage the PromptLearningOptimizer developed here at Arize to improve upon the accuracy of LLMs on classification tasks. Specifically we will be classifying support queries into 30 different classes, including

Account Creation

Login Issues

Password Reset

Two-Factor Authentication

Profile Updates

Billing Inquiry

Refund Request

and 24 more. 

You can view the dataset in datasets/support_queries.csv.

**Note: This notebook `phoenix_support_query_classification.ipynb` complements `support_query_classification.ipynb` by using Phoenix datasets, experiments, and prompt management for Prompt Learning. It's a more end to end way for you to visualize your iterative prompt improvement and see how it performs on train/test sets, and also leverages Phoenix methods for advanced features.**

In [27]:
%pip install arize-phoenix openai pandas

Note: you may need to restart the kernel to use updated packages.


In [28]:
import sys, os, getpass
import openai
import pandas as pd
from openai import AsyncOpenAI
import re
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

In [29]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
openai_client = AsyncOpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [30]:
# Add parent directory to path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# **Setup**

In [31]:
import os
# If you're self-hosting Phoenix, change this value:
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass.getpass('Phoenix Collector Endpoint:')

PHOENIX_API_KEY = getpass.getpass('Phoenix API Key:')
os.environ["PHOENIX_API_KEY"] = PHOENIX_API_KEY

from phoenix.client import Client
phoenix_client = Client()

## **Make train/test sets**

We use an 80/20 train/test split to train our prompt. The optimizer will use the training set to visualize and analyze its errors and successes, and make prompt updates based on these results. We will then test on the test set to see how that prompt performs on unseen data. 

We will be exporting these datasets to Phoenix. In Phoenix you will be able to view the experiments we run on the train/test sets.

In [40]:
data = pd.read_csv("../datasets/support_queries.csv")

train_set = data.sample(frac=0.8, random_state=42)
test_set = data.drop(train_set.index)

train_dataset = phoenix_client.datasets.create_dataset(
        name="training_data_",
        dataframe=train_set,
        input_keys=['query'],
        output_keys=['ground_truth'],
    )

test_dataset = phoenix_client.datasets.create_dataset(
        name="test_data_",
        dataframe=test_set,
        input_keys=['query'],
        output_keys=['ground_truth'],
    )
        

## **Base Prompt for Optimization**

This is our base prompt - our 0th iteration. This is the prompt we will be optimizing for our task.

We also upload our prompt to Phoenix. Phoenix Prompt Hub serves as a repository for your prompts. You will be able to view all iterations of your prompt as its optimized, along with some metrics.

In [41]:
from phoenix.client.types import PromptVersion

system_prompt = """
support query: {query}
Account Creation
Login Issues
Password Reset
Two-Factor Authentication
Profile Updates
Billing Inquiry
Refund Request
Subscription Upgrade/Downgrade
Payment Method Update
Invoice Request
Order Status
Shipping Delay
Product Return
Warranty Claim
Technical Bug Report
Feature Request
Integration Help
Data Export
Security Concern
Terms of Service Question
Privacy Policy Question
Compliance Inquiry
Accessibility Support
Language Support
Mobile App Issue
Desktop App Issue
Email Notifications
Marketing Preferences
Beta Program Enrollment
General Feedback

Return just the category, no other text.
"""

def upload_prompt_phoenix(system_prompt, name, iteration, prompt_versions, train_metric, test_metric):
    prompt_version = PromptVersion(
        [{"role": "system", "content": system_prompt}],  # System message
        model_name="gpt-3.5-turbo",  # Model being used
        description="Prompt for JSON webpage generation",
        model_provider="OPENAI"
    )

    # Create prompt in Phoenix
    initial_prompt_version = phoenix_client.prompts.create(
        name=name,
        version=prompt_version,
    )

    prompt_versions.append({
        "iteration": iteration,
        "prompt": system_prompt,
        "phoenix_id": initial_prompt_version.id if hasattr(initial_prompt_version, 'id') else None,
        "train_metric": train_metric,
        "test_metric": test_metric
    })
    return prompt_versions



## **Output Generator**

This function calls OpenAI with our prompt on every row of our dataset to generate outputs. It leverages llm_generate, a Phoenix function, for concurrency in calling LLMs. 

We return the output column, which contains outputs for every row of our dataset, or every support query in our dataset. 

In [34]:
def generate_task(system_prompt):

    async def output_task(input):
        formatted_prompt = system_prompt.replace("{query}", input.get("query"))
        response = await openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": formatted_prompt}],
        )
        return response.choices[0].message.content
    
    return output_task

In [35]:
def normalize(label):
        return label.strip().strip('"').strip("'").lower()

## **Evaluator**

In this section we define our LLM-as-judge eval. 

Prompt Learning works by generating natural language evaluations on your outputs. These evaluations help guide the prompt optimizer towards building an optimized prompt. 

You should spend time thinking about how to write an informative eval. Your eval makes or breaks this prompt optimizer. With helpful feedback, our prompt optimizer will be able to generate a stronger optimized prompt much more effectively than with sparse or unhelpful feedback. 

Below is a great example for building a strong eval. You can see that we return many evaluations, including
- **correctness**: correct/incorrect - whether the support query was classified correctly or incorrectly.

-  **explanation**: Brief explanation of why the predicted classification is correct or incorrect, referencing the correct label if relevant.

-  **confusion_reason**: If incorrect, explains why the model may have made this choice instead of the correct classification. Focuses on likely sources of confusion. If correct, 'no confusion'.

-  **error_type**: One of: 'broad_vs_specific', 'keyword_bias', 'multi_intent_confusion', 'ambiguous_query', 'off_topic', 'paraphrase_gap', 'other'. Use 'none' if correct. Include the definition of the chosen error type, which are passed into the evaluator's prompt. 

-  **evidence_span**: Exact phrase(s) from the query that strongly indicate the correct classification.

-  **prompt_fix_suggestion**: One clear instruction to add to the classifier prompt to prevent this error.

**Take a look at support_query_classification/evaluator_prompt.txt for the full prompt!**

Our evaluator leverages llm_generate once again to build these llm evals with concurrency. We use an output parser to ensure that our eval is returned in proper json format. 

In [36]:
import re
from phoenix.experiments.types import EvaluationResult

def find_attributes(output):
    patterns = {
        "correctness": r'"correctness":\s*"([^"]*)"',
        "explanation": r'"explanation":\s*"([^"]*)"',
        "confusion_reason": r'"confusion_reason":\s*"([^"]*)"',
        "error_type": r'"error_type":\s*"([^"]*)"',
        "evidence_span": r'"evidence_span":\s*"([^"]*)"',
        "prompt_fix_suggestion": r'"prompt_fix_suggestion":\s*"([^"]*)"'
    }

    return tuple(
        (match := re.search(pattern, output, re.IGNORECASE)) and match.group(1)
        for pattern in patterns.values()
    )


def eval_parser(response: str) -> dict:
    correctness, explanation, confusion_reason, error_type, evidence_span, prompt_fix_suggestion = find_attributes(response)
    return {
        "correctness": correctness,
        "explanation": explanation,
        "confusion_reason": confusion_reason,
        "error_type": error_type,
        "evidence_span": evidence_span,
        "prompt_fix_suggestion": prompt_fix_suggestion
    }


async def output_evaluator(input, expected, output):
    with open("../prompts/support_query_classification/evaluator_prompt.txt", "r") as file:
        evaluator_prompt = file.read()

    evaluator_prompt = evaluator_prompt.replace("{query}", input.get("query"))
    evaluator_prompt = evaluator_prompt.replace("{ground_truth}", expected.get("ground_truth"))
    evaluator_prompt = evaluator_prompt.replace("{output}", output)

    eval_result = await openai_client.chat.completions.create(
        model="gpt-5-2025-08-07",
        messages=[{"role": "user", "content": evaluator_prompt}],
        response_format={"type": "json_object"},
    )

    response = eval_result.choices[0].message.content
    parsed_eval_result = eval_parser(response)
    explanation=f"""correctness: {parsed_eval_result.get("correctness", "")};
        explanation: {parsed_eval_result.get("explanation", "")};
        confusion_reason: {parsed_eval_result.get("confusion_reason", "")};
        error_type: {parsed_eval_result.get("error_type", "")};
        evidence_span: {parsed_eval_result.get("evidence_span", "")};
        prompt_fix_suggestion: {parsed_eval_result.get("prompt_fix_suggestion", "")};"""

    return {
        "score": float(parsed_eval_result.get("correctness") == "correct"),
        "label": parsed_eval_result.get("correctness", ""),
        "explanation": explanation,
    }

async def test_evaluator(expected, output):
    return normalize(expected.get("ground_truth")) == normalize(output)

## Metrics

Below we define some metrics that will compute on each iteration of prompt optimization. It will help us measure how our classifier with the current iteration's prompt performs.

Specifically we use scikit learn for precision, recall, f1 score, and simple accuracy.

In [37]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import requests
def compute_metric(experiment, scorer="accuracy", average="macro"):
    """
    Compute the requested classification metric from a Phoenix experiment.

    Args:
        experiment: an object with an .id field (Phoenix Experiment).
        scorer (str): one of "accuracy", "f1", "precision", "recall".
        average (str): averaging method for multi-class classification.
    
    Returns:
        float: computed metric value.
    """
    print(experiment)
    experiment_id = experiment["experiment_id"]
    url = f"{os.environ['PHOENIX_COLLECTOR_ENDPOINT']}/v1/experiments/{experiment_id}/json"
    headers = {
        "Authorization": f"Bearer {os.environ['PHOENIX_API_KEY']}"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise RuntimeError(f"Failed to fetch experiment data: {response.status_code} {response.text}")

    results = response.json()
    
    y_true = [normalize(entry["reference_output"]["ground_truth"]) for entry in results]
    y_pred = [normalize(entry["output"]) for entry in results]

    if scorer == "accuracy":
        return accuracy_score(y_true, y_pred)
    elif scorer == "f1":
        return f1_score(y_true, y_pred, zero_division=0, average=average)
    elif scorer == "precision":
        return precision_score(y_true, y_pred, zero_division=0, average=average)
    elif scorer == "recall":
        return recall_score(y_true, y_pred, zero_division=0, average=average)
    else:
        raise ValueError(f"Unknown scorer: {scorer}")

## Experiment Processor

This function pulls a Phoenix experiment and loads the data into a pandas dataframe so it can run through the optimizer.

Specifically it:
- Pulls the experiment data from Phoenix
- Adds the input column to the dataframe
- Adds the evals to the dataframe
- Adds the output to the dataframe
- Returns the dataframe

In [38]:
import ast

def process_experiment(experiment, train_set, input_column_name, output_column_name,feedback_columns = None):
    """
    Update existing columns in `train_set` with feedback from experiment annotations.

    Args:
        experiment_json (list): JSON data from experiment.
        train_set (pd.DataFrame): DataFrame that already contains the feedback columns.
        feedback_columns (list): List of feedback field names to update.
    
    Returns:
        pd.DataFrame: Updated DataFrame with values filled in from experiment annotations.
    """

    experiment_id = experiment["experiment_id"]
    url = f"{os.environ['PHOENIX_COLLECTOR_ENDPOINT']}/v1/experiments/{experiment_id}/json"
    headers = {
        "Authorization": f"Bearer {os.environ['PHOENIX_API_KEY']}"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise RuntimeError(f"Failed to fetch experiment data: {response.status_code} {response.text}")

    results = response.json()

    train_set["ground_truth"] = [None] * len(train_set)
    if feedback_columns:
        for col in feedback_columns:
            train_set[col] = [None] * len(train_set)

    train_set = train_set.reset_index(drop=True)

    for i, entry in enumerate(results):
        eval_output = entry["annotations"][0]["explanation"]
        train_set.loc[i, "ground_truth"] = entry["reference_output"]["ground_truth"]
        if feedback_columns:
            for item in eval_output.split(";"):
                key_value = item.split(":")
                if key_value[0].strip() in feedback_columns:
                    key, value = key_value[0].strip(), key_value[1].strip()
                    train_set.loc[i, key] = value

    if "output" in train_set.columns:
        train_set.rename(columns={"output": "ground_truth"}, inplace=True)

    train_set[output_column_name] = [entry.get("output") for entry in results]

    train_set[input_column_name] = [entry.get("input") for entry in results]
    
    return train_set


# Prompt Optimization Loop with Phoenix Experiments

This code implements an iterative prompt optimization system that uses **Phoenix experiments** to evaluate and improve prompts based on feedback from LLM evaluators.


## Overview

The `optimize_loop` function automates prompt engineering by:

- Evaluating prompts using Phoenix experiments  
- Collecting detailed feedback from LLM evaluators  
- Optimizing prompts via a learning-based optimizer  
- Iterating until the performance threshold is met or the loop limit is reached  


## Step-by-Step Breakdown

Each of these numbers are added as comments in the code.

### 1. Initialization

- Set up tracking variables:
  - `train_metrics`, `test_metrics`, `raw_dfs` for storing evaluation results
- Convert training dataset to a DataFrame for easy updates

### 2. Baseline Evaluation

- Run an initial experiment using the **test set**
- Establish a **baseline metric** (e.g., accuracy, F1) to compare against future improvements

### 3. Early Exit Check

- If the **initial prompt already meets the performance threshold**, skip further optimization to save time and compute

### 4. Main Optimization Loop

For each iteration (up to `loops`):

#### 4a. Run Training Experiment

- Execute the current prompt on the **training set**
- Use LLM evaluators to generate **natural language feedback**

#### 4b. Process Feedback

- Extract structured information from evaluator outputs:
  - Correctness
  - Explanation
  - Confusion reason
  - Error type
  - Prompt fix suggestions
- Update the training DataFrame with this feedback

#### 4c. Generate Learning Annotations

- Convert feedback into structured annotations for the optimizer to learn from
- This allows learning from evaluator insights in a consistent format

#### 4d. Optimize the Prompt

- Pass feedback to the **PromptLearningOptimizer**
- Generate an **improved prompt** that attempts to correct issues found in the previous iteration

#### 4e. Evaluate on Test Set

- Evaluate the updated prompt on the **held-out test set**
- Assess **generalization** beyond the training data

#### 4f. Track Metrics

- Log metrics for:
  - Training set performance
  - Test set performance
- Store raw results for further analysis or visualization

#### 4g. Convergence Check

- If the new prompt's test metric **meets or exceeds the threshold**, exit the loop early

In [ ]:
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer
from phoenix.client.experiments import async_run_experiment
import copy
import asyncio

prompt_name = "support_query_classification"

async def optimize_loop(
    train_dataset,
    test_dataset,
    system_prompt,
    evaluators,
    feedback_columns,
    threshold=1,
    loops=5,
    scorer="accuracy",
    prompt_versions=[],
):
    """
    scorer: one of "accuracy", "f1", "precision", "recall"
    """
    curr_loop = 1
    train_metrics = []
    test_metrics = []
    raw_dfs = []
    train_df = train_dataset.to_dataframe()

    print(f"🚀 Starting prompt optimization with {loops} iterations (scorer: {scorer}, threshold: {threshold})")
    
    print(f"�� Initial evaluation:")

    task = generate_task(system_prompt)

    initial_experiment = await async_run_experiment(
        dataset=test_dataset,
        task=task,
        evaluators=[test_evaluator]
    )

    initial_metric_value = compute_metric(initial_experiment, scorer)
    print(f"✅ Initial {scorer}: {initial_metric_value}")

    test_metrics.append(initial_metric_value)
    raw_dfs.append(copy.deepcopy(test_set))

    if initial_metric_value >= threshold:
        print("🎉 Initial prompt already meets threshold!")
        return {
            "train": train_metrics,
            "test": test_metrics,
            "prompt": prompt_versions,
            "raw": raw_dfs
        }
    
    prompt_versions = upload_prompt_phoenix(system_prompt, prompt_name, 0, [], None, initial_metric_value)

    # Initialize all feedback columns

    while loops > 0:
        print(f"📊 Loop {curr_loop}: Optimizing prompt...")
        
        task = generate_task(system_prompt)

        train_experiment = await async_run_experiment(
            dataset=train_dataset,
            task=task,
            evaluators=evaluators,
            concurrency=10
        )

        train_df = process_experiment(train_experiment, train_set, "query", "output", feedback_columns)

        optimizer = PromptLearningOptimizer(
            prompt=system_prompt,
            model_choice="gpt-4o",
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )

        with open("../prompts/support_query_classification/annotations_prompt.txt", "r") as file:
            annotations_prompt = file.read()

        annotations = optimizer.create_annotation(
            system_prompt,
            ["query"],
            train_df,
            feedback_columns,
            [annotations_prompt],
            "output",
            "ground_truth"
        )

        system_prompt = optimizer.optimize(
            train_df,
            "output",
            feedback_columns=feedback_columns,
            context_size_k=90000,
            annotations=annotations,
        )
        train_metric_post_value = compute_metric(train_experiment, scorer)
        train_metrics.append(train_metric_post_value)

        test_experiment = await async_run_experiment(
            dataset=test_dataset,
            task=generate_task(system_prompt),
            evaluators=[test_evaluator]
        )
        test_metric_post_value = compute_metric(test_experiment, scorer)
        test_metrics.append(test_metric_post_value)

        print(f"✅ Train {scorer}: {train_metric_post_value}")
        print(f"✅ Test {scorer}: {test_metric_post_value}")

        prompt_versions = upload_prompt_phoenix(system_prompt, prompt_name, curr_loop, prompt_versions, train_metric_post_value, test_metric_post_value)

        if test_metric_post_value >= threshold:
            print("🎉 Prompt optimization met threshold!")
            break

        loops -= 1
        curr_loop += 1

    return {
        "train": train_metrics,
        "test": test_metrics,
        "prompt": prompt_versions,
        "raw": raw_dfs
    }

# Main execution - use asyncio.run() to run the async function
evaluators = [output_evaluator]
feedback_columns = ["correctness", "explanation", "confusion_reason", "error_type", "evidence_span", "prompt_fix_suggestion"]
result = asyncio.run(optimize_loop(train_dataset, test_dataset, system_prompt, evaluators, feedback_columns, loops=5, scorer="accuracy"))

🚀 Starting prompt optimization with 5 iterations (scorer: accuracy, threshold: 1)
�� Initial evaluation:
🧪 Experiment started.
📺 View dataset experiments: https://app.phoenix.arize.com/s/prompt-opt//datasets/RGF0YXNldDoxMTM=/experiments
🔗 View this experiment: https://app.phoenix.arize.com/s/prompt-opt//datasets/RGF0YXNldDoxMTM=/compare?experimentId=RXhwZXJpbWVudDozMDM=


running tasks |█████████▎| 29/31 (93.5%) | ⏳ 00:07<00:00 |  3.94it/s

✅ Task runs completed.
🧠 Evaluation started.


running tasks |██████████| 31/31 (100.0%) | ⏳ 00:09<00:00 |  3.40it/s


Experiment completed with 31 task runs and 31 evaluation runs
{'experiment_id': 'RXhwZXJpbWVudDozMDM=', 'dataset_id': 'RGF0YXNldDoxMTM=', 'dataset_version_id': 'RGF0YXNldFZlcnNpb246MTEz', 'task_runs': [{'dataset_example_id': 'RGF0YXNldEV4YW1wbGU6NzE3OA==', 'output': 'Account Creation', 'repetition_number': 1, 'start_time': datetime.datetime(2025, 8, 12, 5, 44, 19, 403418, tzinfo=datetime.timezone.utc), 'end_time': datetime.datetime(2025, 8, 12, 5, 44, 19, 403316, tzinfo=datetime.timezone.utc), 'trace_id': '4ee61aac47cdfba1f162e7c3b77d2f75', 'error': None, 'id': 'RXhwZXJpbWVudFJ1bjoxOTQzMQ==', 'experiment_id': 'RXhwZXJpbWVudDozMDM='}, {'dataset_example_id': 'RGF0YXNldEV4YW1wbGU6NzE3OQ==', 'output': 'Billing Inquiry', 'repetition_number': 1, 'start_time': datetime.datetime(2025, 8, 12, 5, 44, 19, 404743, tzinfo=datetime.timezone.utc), 'end_time': datetime.datetime(2025, 8, 12, 5, 44, 19, 404688, tzinfo=datetime.timezone.utc), 'trace_id': 'ebb7e803cba2e1d6fe6e9e1c455cbecb', 'error': None,

...

running tasks |██████████| 31/31 (100.0%) | ⏳ 00:07<00:00 |  5.23it/s

✅ Task runs completed.
🧠 Evaluation started.


running tasks |██████████| 31/31 (100.0%) | ⏳ 00:08<00:00 |  3.68it/s


Experiment completed with 31 task runs and 31 evaluation runs
{'experiment_id': 'RXhwZXJpbWVudDozMjM=', 'dataset_id': 'RGF0YXNldDoxMTM=', 'dataset_version_id': 'RGF0YXNldFZlcnNpb246MTEz', 'task_runs': [{'dataset_example_id': 'RGF0YXNldEV4YW1wbGU6NzE3OA==', 'output': 'Account Creation', 'repetition_number': 1, 'start_time': datetime.datetime(2025, 8, 12, 6, 5, 44, 389590, tzinfo=datetime.timezone.utc), 'end_time': datetime.datetime(2025, 8, 12, 6, 5, 44, 389530, tzinfo=datetime.timezone.utc), 'trace_id': '769980b10d0172521d689a58030a3573', 'error': None, 'id': 'RXhwZXJpbWVudFJ1bjoyMTAwNw==', 'experiment_id': 'RXhwZXJpbWVudDozMjM='}, {'dataset_example_id': 'RGF0YXNldEV4YW1wbGU6NzE3OQ==', 'output': 'Subscription Upgrade/Downgrade', 'repetition_number': 1, 'start_time': datetime.datetime(2025, 8, 12, 6, 5, 44, 390900, tzinfo=datetime.timezone.utc), 'end_time': datetime.datetime(2025, 8, 12, 6, 5, 44, 390850, tzinfo=datetime.timezone.utc), 'trace_id': 'dc969a7f2866c3e6e0a7bd9a5b4518a1', 'er

/opt/anaconda3/envs/base2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
/opt/anaconda3/envs/base2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_pred = type_of_target(y_pred, input_name="y_pred")


# Prompt Optimized!

The code below picks the prompt with the highest score on the test set, and displays the training/test metrics and delta for that prompt.

In [43]:
# Find the best index based on highest test accuracy
best_idx = max(range(len(result["test"])), key=lambda i: result["test"][i])

# Retrieve values
best_prompt = result["prompt"][best_idx - 1]
best_test_acc = result["test"][best_idx]
best_train_acc = result["train"][best_idx - 1] if (best_idx - 1) < len(result["train"]) else None
initial_test_acc = result["test"][0]
initial_train_acc = result["train"][0] if result["train"] else None

# Print results
print("\n🔍 Best Prompt Found:")
print(best_prompt)
print(f"🧪 Initial Test Accuracy: {initial_test_acc}")
print(f"🧪 Optimized Test Accuracy: {best_test_acc} (Δ {best_test_acc - initial_test_acc:.4f})")


🔍 Best Prompt Found:
{'iteration': 2, 'prompt': "You are tasked with classifying user queries into specific support categories based on the primary intent of the query. Use the following refined guidelines and examples to ensure accurate classification:\n\n1. **Contextual Understanding**: Focus on the overall context and primary intent of the query rather than isolated keywords. Consider the user's main concern or request, and assess both direct and indirect evidence in the query.\n\n2. **Prioritization of Intent**: When a query contains multiple potential intents, prioritize the most explicit or primary intent. Use additional context clues to determine the central issue, and consider the aftermath of actions mentioned.\n\n3. **Handling of Informal Language**: Recognize and correctly interpret informal or slang expressions that may indicate a specific category. Be aware of paraphrased or colloquial language that might express the same intent.\n\n4. **Specific vs. Broad Categories**: C